In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor

In [ ]:
data_dir = '/input/'
X_train = pd.read_csv(data_dir + 'train.csv', index_col="Id")
X_test = pd.read_csv(data_dir + 'test.csv', index_col="Id")